In [1]:
import os

import pandas as pd
import math
import random as rd
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import pickle
import dlib

from deepface import DeepFace
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
from deepface.extendedmodels import Age, Gender, Race, Emotion
#from deepface.modules import verification
from deepface.commons import functions, distance as dst

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dropout,LSTM,\
                            TimeDistributed,Flatten,Dense,Bidirectional,ConvLSTM2D,MaxPooling3D,AveragePooling2D,Lambda,\
                            Activation,BatchNormalization

from tensorflow import keras
tf.test.gpu_device_name()

2024-08-08 20:32:59.184086: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


''

In [2]:
seed_constant = 27
np.random.seed(seed_constant)
rd.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [3]:
DIR = '/var/scratch/mpa326/'

In [4]:
import pickle
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_train_50m_50f', 'rb')     
df = pickle.load(dbfile)

y_train_50m_50f = df['labels']
gender_train_50m_50f = df['genders']
X_train_50m_50f=df['features']
paths_train_50m_50f = df['video_files_paths']

In [5]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_train_40m_60f', 'rb')     
df = pickle.load(dbfile)

y_train_40m_60f = df['labels']
gender_train_40m_60f = df['genders']
X_train_40m_60f=df['features']
paths_train_40m_60f = df['video_files_paths']

In [6]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_train_60m_40f', 'rb')     
df = pickle.load(dbfile)

y_train_60m_40f = df['labels']
gender_train_60m_40f = df['genders']
X_train_60m_40f=df['features']
paths_train_60m_40f = df['video_files_paths']

In [7]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_test_common','rb')
df = pickle.load(dbfile)

y_test = df['labels']
gender_test = df['genders']
X_test=df['features']
paths_test = df['video_files_paths']

In [8]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_train_50m_50f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_50m_50f=df['features']

In [9]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_train_40m_60f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_40m_60f=df['features']

In [10]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_train_60m_40f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_60m_40f=df['features']

In [11]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_test_common', 'rb')     
df = pickle.load(dbfile)
X_test_aud=df['features']

In [12]:
from sklearn.utils import shuffle
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, random_state=27)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, random_state=7)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, random_state=42)

In [13]:
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, random_state=27)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, random_state=7)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, random_state=42)

In [14]:
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, random_state=27)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, random_state=7)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, random_state=42)

In [15]:
X_test, X_test_aud, y_test, gender_test = shuffle(X_test, X_test_aud, y_test, gender_test, random_state=27)
X_test, X_test_aud, y_test, gender_test = shuffle(X_test, X_test_aud, y_test, gender_test, random_state=7)
X_test, X_test_aud, y_test, gender_test = shuffle(X_test, X_test_aud, y_test, gender_test, random_state=42)

In [16]:
from tensorflow.keras.models import Model
emotion=Emotion.loadModel()
for layer in emotion.layers:
    print(layer.name)
features_emotion=Model(inputs=emotion.input, outputs=emotion.get_layer('dense_1').output)

conv2d
max_pooling2d
conv2d_1
conv2d_2
average_pooling2d
conv2d_3
conv2d_4
average_pooling2d_1
flatten
dense
dropout
dense_1
dropout_1
dense_2


In [17]:
df_ = np.ones((960,20,48,48))
for i in range(960):
  for j in range(20):
    img=X_train_50m_50f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_50m_50f=df_
df_=[]

In [18]:
df_ = np.ones((960,20,48,48))
for i in range(960):
  for j in range(20):
    img=X_train_40m_60f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_40m_60f=df_
df_=[]

In [19]:
df_ = np.ones((960,20,48,48))
for i in range(960):
  for j in range(20):
    img=X_train_60m_40f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_60m_40f=df_
df_=[]

In [20]:
df_ = np.ones((288,20,48,48))
for i in range(288):
  for j in range(20):
    img=X_test[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_test=df_
df_=[]

In [21]:
features_emotion.trainable=False

In [22]:
class Localization(tf.keras.layers.Layer):
    def __init__(self):
        super(Localization, self).__init__()
        self.pool1 = tf.keras.layers.MaxPool2D()
        self.conv1 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D()
        self.conv2 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(20, activation='relu')
        self.fc2 = tf.keras.layers.Dense(6, activation=None, bias_initializer=tf.keras.initializers.constant([1.0, 0.0, 0.0, 0.0, 1.0, 0.0]), kernel_initializer='zeros')

    def build(self, input_shape):
        print("Building Localization Network with input shape:", input_shape)

    def compute_output_shape(self, input_shape):
        return [None, 2, 3]

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        theta = self.fc2(x)
        theta = tf.keras.layers.Reshape((2, 3))(theta)
        return theta

In [23]:
class BilinearInterpolation(tf.keras.layers.Layer):
    def __init__(self, height=40, width=40):
        super(BilinearInterpolation, self).__init__()
        self.height = height
        self.width = width

    def compute_output_shape(self, input_shape):
        return [None, self.height, self.width, 1]

    def get_config(self):
        return {
            'height': self.height,
            'width': self.width,
        }
    
    def build(self, input_shape):
        print("Building Bilinear Interpolation Layer with input shape:", input_shape)

    def advance_indexing(self, inputs, x, y):
        '''
        Numpy like advance indexing is not supported in tensorflow, hence, this function is a hack around the same method
        '''        
        shape = tf.shape(inputs)
        batch_size, _, _ = shape[0], shape[1], shape[2]
        
        batch_idx = tf.range(0, batch_size)
        batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
        b = tf.tile(batch_idx, (1, self.height, self.width))
        indices = tf.stack([b, y, x], 3)
        return tf.gather_nd(inputs, indices)

    def call(self, inputs):
        images, theta = inputs
        homogenous_coordinates = self.grid_generator(batch=tf.shape(images)[0])
        return self.interpolate(images, homogenous_coordinates, theta)

    def grid_generator(self, batch):
        x = tf.linspace(-1, 1, self.width)
        y = tf.linspace(-1, 1, self.height)
            
        xx, yy = tf.meshgrid(x, y)
        xx = tf.reshape(xx, (-1,))
        yy = tf.reshape(yy, (-1,))
        homogenous_coordinates = tf.stack([xx, yy, tf.ones_like(xx)])
        homogenous_coordinates = tf.expand_dims(homogenous_coordinates, axis=0)
        homogenous_coordinates = tf.tile(homogenous_coordinates, [batch, 1, 1])
        homogenous_coordinates = tf.cast(homogenous_coordinates, dtype=tf.float32)
        return homogenous_coordinates
    
    def interpolate(self, images, homogenous_coordinates, theta):

        with tf.name_scope("Transformation"):
            transformed = tf.matmul(theta, homogenous_coordinates)
            transformed = tf.transpose(transformed, perm=[0, 2, 1])
            transformed = tf.reshape(transformed, [-1, self.height, self.width, 2])
                
            x_transformed = transformed[:, :, :, 0]
            y_transformed = transformed[:, :, :, 1]
                
            x = ((x_transformed + 1.) * tf.cast(self.width, dtype=tf.float32)) * 0.5
            y = ((y_transformed + 1.) * tf.cast(self.height, dtype=tf.float32)) * 0.5

        with tf.name_scope("VariableCasting"):
            x0 = tf.cast(tf.math.floor(x), dtype=tf.int32)
            x1 = x0 + 1
            y0 = tf.cast(tf.math.floor(y), dtype=tf.int32)
            y1 = y0 + 1

            x0 = tf.clip_by_value(x0, 0, self.width-1)
            x1 = tf.clip_by_value(x1, 0, self.width-1)
            y0 = tf.clip_by_value(y0, 0, self.height-1)
            y1 = tf.clip_by_value(y1, 0, self.height-1)
            x = tf.clip_by_value(x, 0, tf.cast(self.width, dtype=tf.float32)-1.0)
            y = tf.clip_by_value(y, 0, tf.cast(self.height, dtype=tf.float32)-1)

        with tf.name_scope("AdvanceIndexing"):
            Ia = self.advance_indexing(images, x0, y0)
            Ib = self.advance_indexing(images, x0, y1)
            Ic = self.advance_indexing(images, x1, y0)
            Id = self.advance_indexing(images, x1, y1)

        with tf.name_scope("Interpolation"):
            x0 = tf.cast(x0, dtype=tf.float32)
            x1 = tf.cast(x1, dtype=tf.float32)
            y0 = tf.cast(y0, dtype=tf.float32)
            y1 = tf.cast(y1, dtype=tf.float32)
                            
            wa = (x1-x) * (y1-y)
            wb = (x1-x) * (y-y0)
            wc = (x-x0) * (y1-y)
            wd = (x-x0) * (y-y0)

            wa = tf.expand_dims(wa, axis=3)
            wb = tf.expand_dims(wb, axis=3)
            wc = tf.expand_dims(wc, axis=3)
            wd = tf.expand_dims(wd, axis=3)
                        
        return tf.math.add_n([wa*Ia + wb*Ib + wc*Ic + wd*Id])

In [24]:
emotions_en = ["neutral", "calm", "happy", "sad", "angry", "fearful", "disgust", "surprised"]

In [25]:
def create__LRCN_with_STN() :
  image = tf.keras.layers.Input(shape=(20,48,48,1))
  theta = TimeDistributed(Localization())(image)
  x = TimeDistributed(BilinearInterpolation(height=48, width=48))([image, theta])
  features=TimeDistributed(features_emotion)(x)
  lstm=Bidirectional(LSTM(200, activation='tanh',input_shape=(20, 1024),dropout=.3))(features)
  out=Dense(len(emotions_en), activation = 'softmax')(lstm)

  return tf.keras.models.Model(inputs=image, outputs=out)


In [26]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [27]:
from sklearn.model_selection import StratifiedKFold

seed = 7
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [30]:
#Audio

In [28]:
def model_alexnet():
  AlexNet = Sequential()

  #1st Convolutional Layer
  AlexNet.add(Conv2D(filters=96, input_shape=(128,128,3), kernel_size=(11,11), strides=(4,4), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #2nd Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #3rd Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #4th Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #5th Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #Passing it to a Fully Connected layer
  AlexNet.add(Flatten())
  AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(Dropout(0.4))

  # #2nd Fully Connected Layer
  # AlexNet.add(Dense(4096))
  # AlexNet.add(BatchNormalization())
  # AlexNet.add(Activation('relu'))
  # #Add Dropout
  # AlexNet.add(Dropout(0.4))

  #3rd Fully Connected Layer
  AlexNet.add(Dense(1000))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  #Add Dropout
  AlexNet.add(Dropout(0.4))

  #Output Layer
  AlexNet.add(Dense(8))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('softmax'))

  return AlexNet 

In [32]:
#LATE FUSION

In [29]:
video_indice = 3
audio_indice = 1

In [30]:
from tensorflow.keras.models import load_model

save_dir = os.path.abspath(DIR)+'/ravdess_50m50f_audio_model/'
model_audio = model_alexnet()
model_audio.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_audio.load_weights(save_dir +"model_"+str(audio_indice)+".h5")

2024-08-08 20:33:56.882661: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.
2024-08-08 20:33:57.736676: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.
2024-08-08 20:33:58.895362: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.
2024-08-08 20:34:41.263607: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.


In [31]:
save_dir = os.path.abspath(DIR)+'/ravdess_50m50f_video_model/'

model_video = create__LRCN_with_STN()
model_video.compile(loss='sparese_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_video.load_weights(save_dir+"model_"+str(video_indice)+".h5")

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


In [32]:
features_video_model=Model(inputs=model_video.input, outputs=model_video.get_layer('bidirectional').output)

In [33]:
features_video=features_video_model.predict(X_train_50m_50f)

2024-08-08 20:35:10.685049: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 176947200 exceeds 10% of free system memory.


30/30 [==============================] - 15s 315ms/step


In [34]:
features_video.shape

(960, 400)

In [35]:
features_audio_model=Model(inputs=model_audio.input, outputs=model_audio.get_layer('dropout_3').output)

In [36]:
features_audio=features_audio_model.predict(X_train_aud_50m_50f)

30/30 [==============================] - 52s 2s/step


In [37]:
features_audio.shape

(960, 1000)

In [38]:
fused_data_train=np.concatenate((features_video,features_audio),axis=1)

In [39]:
print(fused_data_train.shape)
print(fused_data_train)

(960, 1400)
[[-7.4953324e-01  2.6307213e-01 -3.1220677e-01 ...  0.0000000e+00
   1.3266525e+00  0.0000000e+00]
 [ 1.8515849e-01 -1.1131900e-01 -2.6439235e-01 ...  0.0000000e+00
   1.4309967e-01  3.6613369e-01]
 [ 1.3563527e-01  8.5230398e-01 -4.9928925e-04 ...  1.4416392e+00
   3.2065475e-01  1.4489424e-01]
 ...
 [ 5.6693667e-01 -1.6385494e-01 -1.4361385e-03 ...  0.0000000e+00
   1.2457469e+00  2.5211527e+00]
 [-6.3650429e-02  9.0852320e-02 -4.6145919e-04 ...  0.0000000e+00
   2.5686688e+00  2.5723717e+00]
 [-3.0029687e-01 -1.0286822e-01 -4.1443235e-01 ...  1.5334773e+00
   0.0000000e+00  1.3635838e+00]]


In [40]:
features_video_test=features_video_model.predict(X_test)

9/9 [==============================] - 3s 328ms/step


In [41]:
features_audio_test=features_audio_model.predict(X_test_aud)

9/9 [==============================] - 13s 1s/step


In [42]:
fused_data_test = np.concatenate((features_video_test, features_audio_test), axis=1)

In [43]:
fused_data_test.shape

(288, 1400)

In [44]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
fused_data_train = sc.fit_transform(fused_data_train)
fused_data_test = sc.transform(fused_data_test)

In [45]:
from sklearn.decomposition import PCA

def find_n_components(data, variance_threshold=0.95):
    """
    Determine the number of components to retain a specified amount of variance.

    Parameters:
    - data: array-like, shape (n_samples, n_features)
      The data to perform PCA on.
    - variance_threshold: float, optional (default=0.95)
      The amount of variance that needs to be retained. Should be between 0 and 1.

    Returns:
    - n_components: int
      The number of components to retain the specified amount of variance.
    """
    pca = PCA()
    pca.fit(data)
    
    # Calculate cumulative variance
    cumulative_variance = pca.explained_variance_ratio_.cumsum()
    
    # Find the number of components that satisfy the variance threshold
    n_components = (cumulative_variance >= variance_threshold).argmax() + 1
    
    return n_components

# Example usage:
# Assuming fused_data_train is your training dataset
n_components = find_n_components(fused_data_train, variance_threshold=0.95)
print(f"Number of components to retain 95% variance: {n_components}")

# Applying PCA with the determined number of components
lda = PCA(n_components=n_components)
fused_data_train = lda.fit_transform(fused_data_train)
fused_data_test = lda.transform(fused_data_test)

Number of components to retain 95% variance: 201


In [52]:
# from sklearn.decomposition import PCA

# lda = PCA(n_components=800)
# fused_data_train = lda.fit_transform(fused_data_train, y_train_60m_40f)
# fused_data_test = lda.transform(fused_data_test)

In [46]:
fused_data_train.shape

(960, 201)

In [47]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(201)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [48]:
fused_data_train.shape

(960, 201)

In [49]:
VALIDATION_ACCURACY_SPEECH = []
VALIDATION_LOSS_SPEECH = []

save_dir = os.path.abspath(DIR) + '/ravdess_50m50f_latefusion_PCA_n201_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(fused_data_train, y_train_50m_50f):
    model=multi_modal_NN()

    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

    #early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir + get_model_name(fold_var), monitor='val_accuracy',save_best_only=True, mode='max')
# Start training the model.
    LRCN_model_training_history = model.fit(      x = fused_data_train[train_idx],
                                                  y = y_train_50m_50f[train_idx],
                                                  validation_data=(fused_data_train[val_idx], y_train_50m_50f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 4,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb])
    model.load_weights(save_dir + "model_" + str(fold_var) + ".h5")
	
    results = model.evaluate(fused_data_test, y_test)
    results = dict(zip(model.metrics_names, results))
	
    VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
    VALIDATION_LOSS_SPEECH.append(results['loss'])
	
    tf.keras.backend.clear_session()
	
    fold_var += 1
     

Epoch 1/80
192/192 [==============================] - 8s 26ms/step - loss: 1.1327 - accuracy: 0.7109 - val_loss: 0.7163 - val_accuracy: 0.8021
Epoch 2/80
192/192 [==============================] - 2s 12ms/step - loss: 1.0090 - accuracy: 0.8047 - val_loss: 0.7064 - val_accuracy: 0.8177
Epoch 3/80
192/192 [==============================] - 2s 12ms/step - loss: 0.9749 - accuracy: 0.7969 - val_loss: 0.7015 - val_accuracy: 0.7917
Epoch 4/80
192/192 [==============================] - 2s 11ms/step - loss: 0.9091 - accuracy: 0.8086 - val_loss: 0.6610 - val_accuracy: 0.8125
Epoch 5/80
192/192 [==============================] - 2s 12ms/step - loss: 0.8241 - accuracy: 0.8529 - val_loss: 0.6273 - val_accuracy: 0.8385
Epoch 6/80
192/192 [==============================] - 2s 11ms/step - loss: 0.8541 - accuracy: 0.8216 - val_loss: 0.6291 - val_accuracy: 0.8125
Epoch 7/80
192/192 [==============================] - 2s 11ms/step - loss: 0.7647 - accuracy: 0.8620 - val_loss: 0.6420 - val_accuracy: 0.8021

In [ ]:
#Early Fusion

In [51]:
X_train_60m_40f.shape

(960, 20, 48, 48)

In [52]:
X_train_aud_60m_40f.shape

(960, 128, 128, 3)

In [53]:
X_train_flattened = np.reshape(X_train_60m_40f, (960, -1))

X_train_aud_flattened = np.reshape(X_train_aud_60m_40f, (960, -1))
X_train_aud_flattened.shape

(960, 49152)

In [54]:
X_train_flattened.shape

(960, 46080)

In [55]:
X_test_flattened = np.reshape(X_test, (288, -1))

X_test_aud_flattened = np.reshape(X_test_aud, (288, -1))
X_test_aud_flattened.shape

(288, 49152)

In [56]:
from sklearn.preprocessing import StandardScaler
video_scaler = StandardScaler().fit(X_train_flattened)
video_data_standardized = video_scaler.fit_transform(X_train_flattened)
video_data_standardized_test = video_scaler.fit_transform(X_test_flattened)

In [57]:
video_data_standardized_test.shape

(288, 46080)

In [58]:
audio_scaler = StandardScaler().fit(X_train_aud_flattened)
audio_data_standardized = audio_scaler.fit_transform(X_train_aud_flattened)
audio_data_standardized_test = audio_scaler.fit_transform(X_test_aud_flattened)

In [59]:
audio_data_standardized_test.shape

(288, 49152)

In [60]:
from sklearn.decomposition import PCA
pca_video = PCA(n_components=0.95).fit(video_data_standardized)
video_data_pca = pca_video.fit_transform(video_data_standardized)
video_data_pca_test = pca_video.transform(video_data_standardized_test)

In [61]:
video_data_pca.shape

(960, 645)

In [62]:
pca_audio = PCA(n_components=0.95).fit(audio_data_standardized)
audio_data_pca = pca_audio.fit_transform(audio_data_standardized)
audio_data_pca_test = pca_audio.transform(audio_data_standardized_test)

In [63]:
audio_data_pca_test.shape

(288, 313)

In [64]:
concatenated_train_data = np.concatenate((video_data_pca, audio_data_pca), axis=1)

In [65]:
concatenated_train_data.shape

(960, 958)

In [66]:
concatenated_test_data = np.concatenate((video_data_pca_test, audio_data_pca_test), axis=1)

In [67]:
concatenated_test_data.shape

(288, 958)

In [68]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(958)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [69]:
VALIDATION_ACCURACY_SPEECH = []
VALIDATION_LOSS_SPEECH = []

save_dir = os.path.abspath(DIR) + '/ravdess_60m40f_earlyfusion_PCA_n0.95_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(concatenated_train_data, y_train_60m_40f):
    model=multi_modal_NN()

    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

    #early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir + get_model_name(fold_var), monitor='val_accuracy',save_best_only=True, mode='max')
# Start training the model.
    LRCN_model_training_history = model.fit(      x = concatenated_train_data[train_idx],
                                                  y = y_train_60m_40f[train_idx],
                                                  validation_data=(concatenated_train_data[val_idx], y_train_60m_40f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 4,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb])
    model.load_weights(save_dir + "model_" + str(fold_var) + ".h5")
	
    results = model.evaluate(concatenated_test_data, y_test)
    results = dict(zip(model.metrics_names, results))
	
    VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
    VALIDATION_LOSS_SPEECH.append(results['loss'])
	
    tf.keras.backend.clear_session()
	
    fold_var += 1
     

Epoch 1/80
192/192 [==============================] - 9s 34ms/step - loss: 2.0735 - accuracy: 0.2305 - val_loss: 1.5004 - val_accuracy: 0.5677
Epoch 2/80
192/192 [==============================] - 6s 29ms/step - loss: 1.5786 - accuracy: 0.4375 - val_loss: 1.2848 - val_accuracy: 0.5521
Epoch 3/80
192/192 [==============================] - 6s 32ms/step - loss: 1.4038 - accuracy: 0.4909 - val_loss: 1.2341 - val_accuracy: 0.5938
Epoch 4/80
192/192 [==============================] - 6s 31ms/step - loss: 1.3574 - accuracy: 0.4987 - val_loss: 1.1374 - val_accuracy: 0.6615
Epoch 5/80
192/192 [==============================] - 6s 30ms/step - loss: 1.2244 - accuracy: 0.5885 - val_loss: 1.0988 - val_accuracy: 0.6823
Epoch 6/80
192/192 [==============================] - 6s 31ms/step - loss: 1.1255 - accuracy: 0.6523 - val_loss: 1.1017 - val_accuracy: 0.6667
Epoch 7/80
192/192 [==============================] - 6s 32ms/step - loss: 1.0749 - accuracy: 0.6706 - val_loss: 1.1064 - val_accuracy: 0.6146

In [139]:
# def model_alexnet_1d():
#     model = Sequential()

#     # First Fully Connected Layer
#     model.add(Dense(4096, input_shape=(966,)))  # Input shape matches your data
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     model.add(Dropout(0.4))

#     # Second Fully Connected Layer
#     model.add(Dense(4096))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     model.add(Dropout(0.4))

#     # Third Fully Connected Layer
#     model.add(Dense(1000))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     model.add(Dropout(0.4))

#     # Output Layer
#     model.add(Dense(8))  # Number of classes in your output
#     model.add(BatchNormalization())
#     model.add(Activation('softmax'))

#     return model

In [140]:
# model = model_alexnet_1d()

# model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# LRCN_model_training_history = model.fit(          x = concatenated_train_data,
#                                                   y = y_train_50m_50f,
#                                                   validation_data=(concatenated_test_data, y_test),
#                                                   epochs = 80,
#                                                   batch_size = 4,
#                                                   shuffle = True)
                                            

Epoch 1/80
240/240 [==============================] - 52s 196ms/step - loss: 2.0398 - accuracy: 0.2438 - val_loss: 2.0052 - val_accuracy: 0.3924
Epoch 2/80
240/240 [==============================] - 47s 194ms/step - loss: 1.6784 - accuracy: 0.3865 - val_loss: 1.4859 - val_accuracy: 0.4931
Epoch 3/80
240/240 [==============================] - 47s 196ms/step - loss: 1.4388 - accuracy: 0.4854 - val_loss: 1.3159 - val_accuracy: 0.5556
Epoch 4/80
240/240 [==============================] - 47s 195ms/step - loss: 1.2934 - accuracy: 0.5479 - val_loss: 1.2600 - val_accuracy: 0.5660
Epoch 5/80
240/240 [==============================] - 46s 192ms/step - loss: 1.2255 - accuracy: 0.6052 - val_loss: 1.1243 - val_accuracy: 0.6389
Epoch 6/80
183/240 [=====================>........] - ETA: 9s - loss: 1.1469 - accuracy: 0.5997 

KeyboardInterrupt: 

In [141]:
# VALIDATION_ACCURACY_SPEECH = []
# VALIDATION_LOSS_SPEECH = []

# save_dir = os.path.abspath(DIR) + '/ravdess_50m50f_earlyfusion_PCA_n0.95_alexnet_model/'
# fold_var = 1

# for train_idx, val_idx in kfold.split(concatenated_train_data, y_train_50m_50f):
#     model=model_alexnet_1d()

#     model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

#     #early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)

#     checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir + get_model_name(fold_var), monitor='val_accuracy',save_best_only=True, mode='max')
# # Start training the model.
#     LRCN_model_training_history = model.fit(      x = concatenated_train_data[train_idx],
#                                                   y = y_train_50m_50f[train_idx],
#                                                   validation_data=(concatenated_train_data[val_idx], y_train_50m_50f[val_idx]),
#                                                   epochs = 80,
#                                                   batch_size = 4,
#                                                   shuffle = True,
#                                                   callbacks = [checkpoint_cb])
#     model.load_weights(save_dir + "model_" + str(fold_var) + ".h5")
	
#     results = model.evaluate(concatenated_test_data, y_test)
#     results = dict(zip(model.metrics_names, results))
	
#     VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
#     VALIDATION_LOSS_SPEECH.append(results['loss'])
	
#     tf.keras.backend.clear_session()
	
#     fold_var += 1
     

Epoch 1/80
192/192 [==============================] - 44s 212ms/step - loss: 2.0617 - accuracy: 0.2109 - val_loss: 1.9638 - val_accuracy: 0.3281
Epoch 2/80
192/192 [==============================] - 39s 203ms/step - loss: 1.6796 - accuracy: 0.3737 - val_loss: 1.5894 - val_accuracy: 0.4219
Epoch 3/80
192/192 [==============================] - 38s 198ms/step - loss: 1.4862 - accuracy: 0.4362 - val_loss: 1.2205 - val_accuracy: 0.6094
Epoch 4/80
192/192 [==============================] - 36s 186ms/step - loss: 1.3309 - accuracy: 0.5443 - val_loss: 1.2101 - val_accuracy: 0.5677
Epoch 5/80
192/192 [==============================] - 37s 195ms/step - loss: 1.1746 - accuracy: 0.6393 - val_loss: 1.2139 - val_accuracy: 0.5938
Epoch 6/80
192/192 [==============================] - 37s 191ms/step - loss: 1.1388 - accuracy: 0.6302 - val_loss: 1.2338 - val_accuracy: 0.5573
Epoch 7/80
192/192 [==============================] - 39s 204ms/step - loss: 1.0567 - accuracy: 0.7044 - val_loss: 1.1329 - val_ac

In [ ]:
#Intermediate Fusion

In [132]:
from tensorflow.keras.models import load_model
audio_indice = 1
save_dir = os.path.abspath(DIR)+'/ravdess_60m40f_audio_model/'
model_audio = model_alexnet()
model_audio.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_audio.load_weights(save_dir +"model_"+str(audio_indice)+".h5")

In [133]:
save_dir = os.path.abspath(DIR)+'/ravdess_60m40f_video_model/'
video_indice = 5
model_video = create__LRCN_with_STN()
model_video.compile(loss='sparese_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_video.load_weights(save_dir+"model_"+str(video_indice)+".h5")

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


In [136]:
features_video_model=Model(inputs=model_video.input, outputs=model_video.get_layer('time_distributed_2').output)

In [137]:
features_video=features_video_model.predict(X_train_60m_40f)

30/30 [==============================] - 14s 322ms/step


In [138]:
features_video.shape

(960, 20, 1024)

In [139]:
X_train_flattened = np.reshape(features_video, (960, -1))

In [140]:
X_train_flattened.shape

(960, 20480)

In [141]:
features_video_test = features_video_model.predict(X_test)

9/9 [==============================] - 3s 304ms/step


In [142]:
features_video_test.shape

(288, 20, 1024)

In [143]:
X_test_flattened = np.reshape(features_video_test, (288, -1))

In [144]:
features_audio_model=Model(inputs=model_audio.input, outputs=model_audio.get_layer('flatten').output)

In [145]:
features_audio=features_audio_model.predict(X_train_aud_60m_40f)

30/30 [==============================] - 17s 532ms/step


In [146]:
features_audio.shape

(960, 262144)

In [147]:
features_audio_test = features_audio_model.predict(X_test_aud)

9/9 [==============================] - 5s 592ms/step


In [148]:
features_audio_test.shape

(288, 262144)

In [149]:
from sklearn.preprocessing import StandardScaler
video_scaler = StandardScaler().fit(X_train_flattened)
video_data_standardized = video_scaler.fit_transform(X_train_flattened)
video_data_standardized_test = video_scaler.fit_transform(X_test_flattened)

In [150]:
audio_scaler = StandardScaler().fit(features_audio)
audio_data_standardized = audio_scaler.fit_transform(features_audio)
audio_data_standardized_test = audio_scaler.fit_transform(features_audio_test)

In [151]:
from sklearn.decomposition import PCA
pca_video = PCA(n_components=0.95).fit(video_data_standardized)
video_data_pca = pca_video.fit_transform(video_data_standardized)
video_data_pca_test = pca_video.transform(video_data_standardized_test)

In [152]:
video_data_pca.shape

(960, 618)

In [153]:
video_data_pca_test.shape

(288, 618)

In [154]:
pca_audio = PCA(n_components=0.95).fit(audio_data_standardized)
audio_data_pca = pca_audio.fit_transform(audio_data_standardized)
audio_data_pca_test = pca_audio.transform(audio_data_standardized_test)

In [155]:
audio_data_pca.shape

(960, 474)

In [156]:
audio_data_pca_test.shape

(288, 474)

In [157]:
concatenated_train_data = np.concatenate((video_data_pca, audio_data_pca), axis=1)

In [158]:
concatenated_train_data.shape

(960, 1092)

In [159]:
concatenated_test_data = np.concatenate((video_data_pca_test, audio_data_pca_test), axis=1)

In [160]:
concatenated_test_data.shape

(288, 1092)

In [161]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(1092)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [ ]:
VALIDATION_ACCURACY_SPEECH = []
VALIDATION_LOSS_SPEECH = []

save_dir = os.path.abspath(DIR) + '/ravdess_60m40f_intermediatefusion_PCA_n0.95_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(concatenated_train_data, y_train_60m_40f):
    model=multi_modal_NN()

    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

    #early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir + get_model_name(fold_var), monitor='val_accuracy',save_best_only=True, mode='max')
# Start training the model.
    LRCN_model_training_history = model.fit(      x = concatenated_train_data[train_idx],
                                                  y = y_train_40m_60f[train_idx],
                                                  validation_data=(concatenated_train_data[val_idx], y_train_60m_40f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 4,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb])
    model.load_weights(save_dir + "model_" + str(fold_var) + ".h5")
	
    results = model.evaluate(concatenated_test_data, y_test)
    results = dict(zip(model.metrics_names, results))
	
    VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
    VALIDATION_LOSS_SPEECH.append(results['loss'])
	
    tf.keras.backend.clear_session()
	
    fold_var += 1
     

Epoch 1/80
192/192 [==============================] - 10s 36ms/step - loss: 1.7833 - accuracy: 0.3294 - val_loss: 1.0982 - val_accuracy: 0.6146
Epoch 2/80
192/192 [==============================] - 6s 32ms/step - loss: 1.3645 - accuracy: 0.5286 - val_loss: 0.9882 - val_accuracy: 0.6667
Epoch 3/80
192/192 [==============================] - 7s 34ms/step - loss: 1.2017 - accuracy: 0.6094 - val_loss: 0.9689 - val_accuracy: 0.7083
Epoch 4/80
192/192 [==============================] - 6s 30ms/step - loss: 1.1953 - accuracy: 0.6133 - val_loss: 0.9512 - val_accuracy: 0.6927
Epoch 5/80
192/192 [==============================] - 7s 37ms/step - loss: 1.0859 - accuracy: 0.6641 - val_loss: 0.9256 - val_accuracy: 0.7396
Epoch 6/80
192/192 [==============================] - 6s 32ms/step - loss: 1.0269 - accuracy: 0.7122 - val_loss: 0.8772 - val_accuracy: 0.7500
Epoch 7/80
192/192 [==============================] - 7s 36ms/step - loss: 0.9731 - accuracy: 0.7227 - val_loss: 0.8552 - val_accuracy: 0.750

In [ ]:
#IntermediateFusion Variant 2 **Not Selected**

In [64]:
features_video_model=Model(inputs=model_video.input, outputs=model_video.get_layer('time_distributed_1').output)

In [65]:
features_video=features_video_model.predict(X_train_50m_50f)

30/30 [==============================] - 5s 92ms/step


In [67]:
features_video.shape

(960, 20, 48, 48, 1)

In [68]:
X_train_flattened = np.reshape(features_video, (960, -1))

In [69]:
X_train_flattened.shape

(960, 46080)

In [70]:
features_video_test = features_video_model.predict(X_test)

9/9 [==============================] - 1s 134ms/step


In [71]:
X_test_flattened = np.reshape(features_video_test, (288, -1))

In [72]:
features_audio_model=Model(inputs=model_audio.input, outputs=model_audio.get_layer('flatten_1').output)

In [73]:
features_audio=features_audio_model.predict(X_train_aud_50m_50f)

30/30 [==============================] - 17s 552ms/step


In [74]:
features_audio.shape

(960, 262144)

In [75]:
features_audio_test = features_audio_model.predict(X_test_aud)

9/9 [==============================] - 5s 466ms/step


In [76]:
features_audio_test.shape

(288, 262144)

In [77]:
from sklearn.preprocessing import StandardScaler
video_scaler = StandardScaler().fit(X_train_flattened)
video_data_standardized = video_scaler.fit_transform(X_train_flattened)
video_data_standardized_test = video_scaler.fit_transform(X_test_flattened)

In [78]:
audio_scaler = StandardScaler().fit(features_audio)
audio_data_standardized = audio_scaler.fit_transform(features_audio)
audio_data_standardized_test = audio_scaler.fit_transform(features_audio_test)

In [79]:
from sklearn.decomposition import PCA
pca_video = PCA(n_components=0.95).fit(video_data_standardized)
video_data_pca = pca_video.fit_transform(video_data_standardized)
video_data_pca_test = pca_video.transform(video_data_standardized_test)

In [80]:
video_data_pca.shape

(960, 596)

In [81]:
pca_audio = PCA(n_components=0.95).fit(audio_data_standardized)
audio_data_pca = pca_audio.fit_transform(audio_data_standardized)
audio_data_pca_test = pca_audio.transform(audio_data_standardized_test)

In [82]:
audio_data_pca.shape

(960, 511)

In [83]:
audio_data_pca_test.shape

(288, 511)

In [84]:
concatenated_train_data = np.concatenate((video_data_pca, audio_data_pca), axis=1)

In [85]:
concatenated_train_data.shape

(960, 1107)

In [86]:
concatenated_test_data = np.concatenate((video_data_pca_test, audio_data_pca_test), axis=1)

In [87]:
concatenated_test_data.shape

(288, 1107)

In [88]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(1107)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [89]:
VALIDATION_ACCURACY_SPEECH = []
VALIDATION_LOSS_SPEECH = []

save_dir = os.path.abspath(DIR) + '/ravdess_50m50f_intermediatefusion_2ndVariant_PCA_n0.95_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(concatenated_train_data, y_train_50m_50f):
    model=multi_modal_NN()

    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

    #early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir + get_model_name(fold_var), monitor='val_accuracy',save_best_only=True, mode='max')
# Start training the model.
    LRCN_model_training_history = model.fit(      x = concatenated_train_data[train_idx],
                                                  y = y_train_50m_50f[train_idx],
                                                  validation_data=(concatenated_train_data[val_idx], y_train_50m_50f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 4,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb])
    model.load_weights(save_dir + "model_" + str(fold_var) + ".h5")
	
    results = model.evaluate(concatenated_test_data, y_test)
    results = dict(zip(model.metrics_names, results))
	
    VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
    VALIDATION_LOSS_SPEECH.append(results['loss'])
	
    tf.keras.backend.clear_session()
	
    fold_var += 1
     

Epoch 1/80
192/192 [==============================] - 9s 32ms/step - loss: 1.8559 - accuracy: 0.3112 - val_loss: 1.2366 - val_accuracy: 0.5781
Epoch 2/80
192/192 [==============================] - 7s 36ms/step - loss: 1.3523 - accuracy: 0.5052 - val_loss: 1.0969 - val_accuracy: 0.6406
Epoch 3/80
192/192 [==============================] - 7s 34ms/step - loss: 1.2434 - accuracy: 0.5703 - val_loss: 1.0729 - val_accuracy: 0.6615
Epoch 4/80
192/192 [==============================] - 7s 35ms/step - loss: 1.1398 - accuracy: 0.6549 - val_loss: 1.0545 - val_accuracy: 0.6875
Epoch 5/80
192/192 [==============================] - 7s 38ms/step - loss: 1.0559 - accuracy: 0.6784 - val_loss: 0.9998 - val_accuracy: 0.7188
Epoch 6/80
192/192 [==============================] - 6s 29ms/step - loss: 1.0606 - accuracy: 0.6888 - val_loss: 0.9707 - val_accuracy: 0.7135
Epoch 7/80
192/192 [==============================] - 7s 35ms/step - loss: 0.9805 - accuracy: 0.7188 - val_loss: 0.9556 - val_accuracy: 0.7188